In [ ]:
from utils import DB, getExamplesSorted, Example, BenchmarkResult
from typing import Iterator
from IPython.display import display, Markdown, clear_output
import re

reviewTemplate = """\

#### {title}
{description}

<table>
<tr>
<td>

`slow`:

```cpp
{codeSlow}
```

</td>
<td>

`agent`:

```cpp
{codeAgent}
```

</td>
<td>

`fast`:

```cpp
{codeFast}
```

</td>
</tr>
</table>
"""

includeRegex = re.compile(r"#include.*?\n")


def evaluateResult(testNum: int, model: str):
    stem = f"test{testNum}.{model}.task"
    query = (
        f"for benchmark in {DB.benchmarks}",
        f'  filter benchmark.filename like "{stem}%"',
        f'  let taskNum = to_number(regex_replace(benchmark.filename, "{stem}", ""))',
        "  sort taskNum asc",
        '  return merge(unset(benchmark, "_key", "_id", "_rev"), {"taskNum" : taskNum})',
    )
    benchmarks = DB.get().aql.execute("\n".join(query))
    examples = getExamplesSorted()

    def zipper() -> Iterator[tuple[int, dict, Example]]:
        yield from zip(range(1, 51), benchmarks, examples, strict=True)

    for taskNum, benchResultDoc, example in zipper():
        assert (taskNum == benchResultDoc.pop("taskNum")) and (
            taskNum == int(example._key)
        )
        benchResult = BenchmarkResult(**benchResultDoc)
        codeAgent = benchResult.code
        codeAgent = re.sub(includeRegex, "", codeAgent)  # strip includes
        codeAgent = (
            "\n".join(
                filter(
                    lambda line: line.strip(),
                    codeAgent.split("\n"),
                )
            )
        ).strip()  # strip empty lines

        mdStr = reviewTemplate.format(
            title=example.title,
            description=example.description,
            codeSlow=example.codeSlow.strip(),
            codeFast=example.codeFast.strip(),
            codeAgent=codeAgent,
        )
        display(Markdown(mdStr))
        yield
        input()
        clear_output()

In [12]:
it = evaluateResult(1, "claude-sonnet-4")

In [13]:
next(it)

<table>
<tr>
<td>

`slow`:

```cpp
// inefficient version
#include <iostream>
#include <string>

int main() {
    std::string result;
    for (int i = 0; i < 10000; i++) {
        result = result + std::to_string(i) + " ";
    }
    std::cout << result.substr(0, 20) << "..." << std::endl;
    return 0;
}

```

</td>
<td>

`agent`:

```cpp



// Task 1
  {
    // Optimization: Use stringstream for more efficient string concatenation
    std::ostringstream oss;
    for (int i = 0; i < 10000; i++) {
      oss << i << " ";
    }
    std::string result = oss.str();
    std::cout << result.substr(0, 20) << "..." << std::endl;
  }

```

</td>
<td>

`fast`:

```cpp
// efficient version
#include <iostream>
#include <string>

int main() {
    std::string result;
    result.reserve(70000);  // Approximate size needed
    for (int i = 0; i < 10000; i++) {
        result += std::to_string(i) + " ";
    }
    std::cout << result.substr(0, 20) << "..." << std::endl;
    return 0;
}

```

</td>
</tr>
</table>
